In [1]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 5.9 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
from google.colab.patches import cv2_imshow
import cv2
display.clear_output()
!yolo mode=checks

In [ ]:
!yolo settings datasets_dir=/content/

In [ ]:
!pip install -U torch sahi yolov8 ultralytics numpy opencv-python

In [5]:
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.prediction import visualize_object_predictions
from IPython.display import Image
from numpy import asarray
import cv2

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/AI/R-CNN_proj/innopolis-high-voltage-challenge_1.zip -d my_data

In [10]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
def get_predictions(directory_path, save_path) -> list:
    model_path = f'/content/drive/MyDrive/AI/R-CNN_proj/Danila/best700.pt'

    detection_model = AutoDetectionModel.from_pretrained(
        model_type='yolov8',
        model_path=model_path,
        confidence_threshold=0.35,
        device='cuda:0'
    )

    res = []
    files_and_folders = os.listdir(directory_path)
    jpg_files = [file for file in files_and_folders if file.lower().endswith('.jpg')]

    for item in jpg_files:
        s = f'{directory_path}/{item}'
        result = get_sliced_prediction(s,
                                       detection_model,
                                       slice_height=350,
                                       slice_width=350,
                                       overlap_height_ratio=0.55,
                                       overlap_width_ratio=0.55,
                                       )

        result.export_visuals(save_path, file_name=item)

        size = [result.image_width, result.image_height]
        list_with_coordinates = result.object_prediction_list
        if len(list_with_coordinates) > 0:
            for k, i in enumerate(list_with_coordinates):
                bbox = i.bbox
                x, y, w, h = bbox.to_xywh()
                zz = [x / size[0], y / size[1], w / size[0], h / size[1]]
                zz[0] = zz[0] + zz[2] / 2
                zz[1] = zz[1] + zz[3] / 2
                zz.append(i.score.value)
                zz.append(item[:-4] + '_' + str(k + 1))
                res.append(zz)
        else:
            res.append([0, 0, 0, 0, 0, item])

    return res

In [ ]:
directory_path = '/content/my_data_d'
save_path = '/content/sample_data_d'
res = get_predictions(directory_path, save_path)

In [12]:
import pandas as pd

In [13]:
def save_to_csv(res: list, save_path: str):
    save_path = '/content/sample_data_d'
    d = {}
    for i in res:
        x, y, w, h, prob, name = i
        name = name[:-2]
        if name not in d:
            d[name] = [[x, y, w, h, prob]]
        else:
            d[name].append([x, y, w, h, prob])

    bit = []
    for k, v in d.items():
        sub = [k]
        nex = []
        prob = []
        for i in v:
            if i[0] == 0 and i[1] == 0 and i[2] == 0 and i[3] == 0:
                nex.append(0)
            else:
                nex.append([i[0], i[1], i[2], i[3]])
            prob.append(1 if i[4] > 0 else 0)  # i[4]
        sub.append(nex)
        sub.append(prob)
        bit.append(sub)

    rs = list(zip(*bit))
    percentile_list = pd.DataFrame(
        {'file_name': rs[0],
         'rbbox': rs[1],
         'probability': rs[2],
         })

    percentile_list.to_csv(save_path+"/result.csv", index=False)

In [14]:
save_to_csv(res, '/content/sample_data_d')

In [9]:
model_path = f'/content/drive/MyDrive/AI/R-CNN_proj/Danila/best700.pt'

detection_model = AutoDetectionModel.from_pretrained(
        model_type='yolov8',
        model_path=model_path,
        confidence_threshold=0.35,
        device='cuda:0'
)

In [18]:
res = []
directory_path = '/content/my_data'
files_and_folders = os.listdir(directory_path)
jpg_files = [file for file in files_and_folders if file.lower().endswith('.jpg')]
print(jpg_files)
save_path = '/content/sample_data_2'
for item in jpg_files:
    s = f'{directory_path}/{item}'
    result = get_sliced_prediction(s,
                                   detection_model,
                                    slice_height=500,
                                    slice_width=500,
                                    overlap_height_ratio=0.55,
                                    overlap_width_ratio=0.55,
                                    )

    result.export_visuals(save_path, file_name=item)


['DJI_0038.JPG', 'DJI_0113.JPG', 'DJI_0114.JPG', 'DJI_0120.JPG', 'DJI_0097.JPG', 'DJI_0119.JPG', 'DJI_0111.JPG', 'DJI_0063.JPG', 'DJI_0108.JPG', 'DJI_0059.JPG', 'DJI_0058.JPG', 'DJI_0044.JPG', 'DJI_0095.JPG', 'DJI_0069.JPG', 'DJI_0107.JPG', 'DJI_0116.JPG', 'DJI_0121.JPG', 'DJI_0098.JPG', 'DJI_0105.JPG', 'DJI_0103.JPG', 'DJI_0110.JPG', 'DJI_0096.JPG', 'DJI_0049.JPG', 'DJI_0050.JPG', 'DJI_0094.JPG', 'DJI_0056.JPG', 'DJI_0117.JPG', 'DJI_0081.JPG', 'DJI_0039.JPG', 'DJI_0115.JPG']
Performing prediction on 153 number of slices.


KeyboardInterrupt: ignored

In [ ]:
size = [result.image_width, result.image_height]
        list_with_coordinates = result.object_prediction_list
        if len(list_with_coordinates) > 0:
            for k, i in enumerate(list_with_coordinates):
                bbox = i.bbox
                x, y, w, h = bbox.to_xywh()
                zz = [x / size[0], y / size[1], w / size[0], h / size[1]]
                zz[0] = zz[0] + zz[2] / 2
                zz[1] = zz[1] + zz[3] / 2
                zz.append(i.score.value)
                zz.append(item[:-4] + '_' + str(k + 1))
                res.append(zz)
        else:
            res.append([0, 0, 0, 0, 0, item])

In [ ]:
file_path = "/content/drive/MyDrive/AI/R-CNN_proj/Danila/best700.pt"

In [ ]:
yolov8_model_path = file_path

In [ ]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=yolov8_model_path,
    confidence_threshold=0.0,
    device='cuda:0'
)

Просматриваем разметку на картинках

In [6]:
!unzip /content/drive/MyDrive/AI/RCNN-1/Danila/innopolis-high-voltage-challenge.zip -d test_data

unzip:  cannot find or open /content/drive/MyDrive/AI/RCNN-1/Danila/innopolis-high-voltage-challenge.zip, /content/drive/MyDrive/AI/RCNN-1/Danila/innopolis-high-voltage-challenge.zip.zip or /content/drive/MyDrive/AI/RCNN-1/Danila/innopolis-high-voltage-challenge.zip.ZIP.


In [ ]:

s = f'/content/drive/MyDrive/AI/R-CNN_proj/Danila/DJI_20231010100534_0333_Zenmuse-L1-mission.JPG'
result = get_sliced_prediction(s,
                        detection_model,
                        slice_height=500,
                        slice_width=500,
                        overlap_height_ratio=0.8,
                        overlap_width_ratio=0.8,
                        )


Performing prediction on 1683 number of slices.


In [ ]:
result.export_visuals(f'demo_data2/', file_name='last_step')

In [ ]:
%cp -av "/content/demo_data2" "/content/drive/MyDrive/AI"

'/content/demo_data2/last_step.png' -> '/content/drive/MyDrive/AI/demo_data2/last_step.png'


In [ ]:
directory_path = '/content/test_data'
# Получаем список всех файлов и папок в директории
res = []
files_and_folders = os.listdir(directory_path)
jpg_files = [file for file in files_and_folders if file.lower().endswith('.jpg')]
print(jpg_files)
for item in jpg_files:
    s = f'{directory_path}/{item}'
    result = get_sliced_prediction(s,
                        detection_model,
                        slice_height=500,
                        slice_width=500,
                        overlap_height_ratio=0.55,
                        overlap_width_ratio=0.55,
                        )
    result.export_visuals(f'demo_data2/', file_name=item)

    # img = cv2.imread(f'/content/demo_data/{item}/prediction_visual.png')
    #cv2_imshow(img)

%cp -av "/content/demo_data2" "/content/drive/MyDrive/AI"

## Test

In [ ]:
import os
import tqdm

directory_path = '/content/test_data'
# Получаем список всех файлов и папок в директории
res = []
files_and_folders = os.listdir(directory_path)
jpg_files = [file for file in files_and_folders if file.lower().endswith('.jpg')]

for item in jpg_files:
    s = f'{directory_path}/{item}'
    result = get_sliced_prediction(s,
                        detection_model,
                        slice_height=500,
                        slice_width=500,
                        overlap_height_ratio=0.55,
                        overlap_width_ratio=0.55,
                        )

    size = [result.image_width, result.image_height]
    list_with_coordinates = result.object_prediction_list
    for k, i in enumerate(list_with_coordinates):
       bbox = i.bbox
       x, y, w, h = bbox.to_xywh()
       zz = [x/size[0], y/size[1], w/size[0], h/size[1]]
       zz[0] = zz[0] + zz[2]/2
       zz[1] = zz[1] + zz[3]/2
       zz.append(i.score.value)
       zz.append(item[:-4] +'_'+ str(k+1))
       res.append(zz)
       print(bbox)
print(res)



In [ ]:
d = {}
for i in res:
  x, y, w, h, prob, name = i
  name = name[:-2]
  if name not in d:
    d[name] = [[x, y, w, h, prob]]
  else:
    d[name].append([x, y, w, h, prob])

bit = []
for k, v in d.items():
  sub = [k]
  nex = []
  prob = []
  for i in v:
    nex.append([i[0], i[1],i[2],i[3]])
    prob.append(1) #i[4]
  sub.append(nex)
  sub.append(prob)
  bit.append(sub)

import pandas as pd
rs = list(zip(*bit))
percentile_list = pd.DataFrame(
    {'file_name': rs[0],
     'rbbox': rs[1],
     'probability': rs[2],
    })
df = pd.read_csv('/content/example_submission_final.csv')
df = df.drop([i for i in df.columns if i != 'file_name'], axis=1)
commit = df.merge(percentile_list, how='left').fillna(0)
commit.to_csv('first.csv', index=False)